créditos:https://colab.research.google.com/drive/1FvBBw2JNXHy9WeJusI2PPr2PMnGVbbGM#scrollTo=GBotcBrgqR30 (lembrar de referenciar)

In [2]:
#%matplotlib inline
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#carregando o dataset
data_set = '/content/drive/My Drive/WESAD/'
s2_path = data_set + 'S2.pkl'
#modificação necessária pois os dados estavam em py2 e o ambiente usa py3
with open(s2_path, 'rb') as file:
    s2_data = pickle.load(file, encoding='latin1')

In [5]:
#Separando dados de peito de dados de pulso (apenas dados de peito serão considerados conforme análise dos dados)
c_ax=s2_data['signal']['chest']['ACC'][0:,0]
c_ay=s2_data['signal']['chest']['ACC'][0:,1]
c_az=s2_data['signal']['chest']['ACC'][0:,2]
c_ecg=s2_data['signal']['chest']['ECG'][:,0]
c_emg=s2_data['signal']['chest']['EMG'][:,0]
c_eda=s2_data['signal']['chest']['EDA'][:,0]
c_temp=s2_data['signal']['chest']['Temp'][:,0]
c_resp=s2_data['signal']['chest']['Resp'][:,0]
label=s2_data['label']

# Alterar o objeto de dicionário para um array numpy e o array para um dataframe

*(referenciar)*
**Python | Numpy ndarray.T**
https://www.geeksforgeeks.org/python-numpy-ndarray-t/

**Gerando e Manipulando Arrays em Python**
https://towardsdatascience.com/numpy-array-cookbook-generating-and-manipulating-arrays-in-python-2195c3988b09

In [ ]:
peito = [c_ax, c_ay, c_az, c_ecg, c_eda, c_emg, c_resp, c_temp, label]
p_array = np.array(peito)
p_array = p_array.T #(transposição matriz)
Colunas = ['c_ax', 'c_ay', 'c_az', 'c_ecg', 'c_eda', 'c_emg', 'c_resp', 'c_temp', 'label']

p_df = pd.DataFrame(p_array, columns = Colunas)
p_df

In [ ]:
p_df.describe().T

Identificando os rótulos, usando apenas:
neutro: 0
stress: 1
diversão: 2

Os demais serão desconsiderados nesse estudo

In [32]:
neutro = p_df.loc[p_df['label'] == 1]
stress = p_df.loc[p_df['label'] == 2]


p_loc = pd.concat([neutro, stress])
p_loc

,c_ax,c_ay,c_az,c_ecg,c_eda,c_emg,c_resp,c_temp,label
214583,0.8914,-0.1102,-0.2576,0.030945,5.710983,-0.003708,1.191711,29.083618,1.0
214584,0.8926,-0.1086,-0.2544,0.033646,5.719376,-0.014145,1.139832,29.122437,1.0
214585,0.8930,-0.1094,-0.2580,0.033005,5.706406,0.010208,1.141357,29.115234,1.0
214586,0.8934,-0.1082,-0.2538,0.031815,5.712509,0.012634,1.155090,29.126709,1.0
214587,0.8930,-0.1096,-0.2570,0.030350,5.727005,0.002060,1.133728,29.100861,1.0
...,...,...,...,...,...,...,...,...,...
2021977,0.8760,-0.1346,-0.3020,0.001328,3.075409,-0.004715,-1.161194,31.163483,2.0
2021978,0.8770,-0.1352,-0.2998,0.003433,3.059769,-0.003937,-1.124573,31.090057,2.0
2021979,0.8806,-0.1334,-0.2978,0.003616,3.078079,-0.009018,-1.097107,31.098877,2.0
2021980,0.8778,-0.1314,-0.2964,0.003525,3.058624,0.004944,-1.103210,31.101807,2.0


# Normalizando dados: train test

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_axis.html (referenciar)

In [33]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.utils import to_categorical

In [34]:
x = p_loc.drop('label',axis=1).values
y = to_categorical(p_loc['label'])

scaler = MinMaxScaler(feature_range= (0,1))
x = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape)
print(y_test.shape)
evalSet = [(x_train, y_train), (x_test, y_test)]

(985040, 8)
(246260, 3)


In [35]:
from keras.models import Sequential
from keras. layers import Activation
from keras.layers.core import Dense

In [36]:
model = Sequential()
model.add(Dense(8, input_shape=(8,), activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 63        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 24        
Total params: 159
Trainable params: 159
Non-trainable params: 0
_________________________________________________________________


In [37]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.utils import to_categorical
from keras.optimizers import SGD

In [38]:
model.compile(Adam(lr=.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
model.fit(x_train, y_train, validation_split=0.1, batch_size=10, epochs=3, shuffle=True, verbose=2)

Epoch 1/3
88654/88654 - 86s - loss: 0.0799 - accuracy: 0.9744 - val_loss: 0.0137 - val_accuracy: 0.9960
Epoch 2/3
88654/88654 - 85s - loss: 0.0101 - accuracy: 0.9967 - val_loss: 0.0069 - val_accuracy: 0.9976
Epoch 3/3
88654/88654 - 83s - loss: 0.0049 - accuracy: 0.9982 - val_loss: 0.0030 - val_accuracy: 0.9990


In [41]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# realizar as previsões no dataset de teste
y_pred = model.predict(x_test)

# ver acurácia geral
print('[Acurácia] Neural:', accuracy_score(y_test, y_pred))

# imprimir o classification report
print('\n[Classification Report] Neural')
print( classification_report(y_test, y_pred) )

ValueError: ignored